In [1]:
# Adding time function
import time

import pandas as pd


def timer(func):
    def func_in():
        start_time = time.time()
        func()
        end_time = time.time()
        spend_time = (end_time - start_time)/60
        print("Spend_time:{} min".format(spend_time))
    return func_in

In [2]:
from finrl.meta.data_processor import DataProcessor
data_source="forex"
start_date="2022-11-01"
end_date="2022-11-05"
time_interval = "MINUTE"
ticker_list=["EURUSD.MT4"]
dp = DataProcessor(data_source)
df = dp.download_data(ticker_list, start_date,end_date,time_interval)
# df=df.rename(columns={"time":"Date", "open":"Open", "high": "High", "low":"Low", "close":"Close"})
# df.set_index("Date", inplace=True, drop=True)

Shape of DataFrame:  (5693, 6)

download_data cost time 0.4283413887023926 s



In [3]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf
# %matplotlib inline

In [4]:
df = df.iloc[0:1000]

In [ ]:
mpf.plot(df,type='candle',mav=(2, 5, 10), volume=False)

In [6]:
exp12 = df['Close'].ewm(span=12, adjust=False).mean()
exp26 = df['Close'].ewm(span=26, adjust=False).mean()
macd = exp12 - exp26
signal = macd.ewm(span=9, adjust=False).mean()
histogram = macd - signal

In [27]:
add_plot = [
            mpf.make_addplot(exp12, type='line', color='y'),
            mpf.make_addplot(exp26, type='line', color='r'),
            mpf.make_addplot(histogram, type='bar', width=0.7, panel=1, color='dimgray', alpha=1,
                                    secondary_y=False),
            mpf.make_addplot(macd, panel=1, color='fuchsia', secondary_y=True),
            mpf.make_addplot(signal, panel=1, color='b', secondary_y=True),
            mpf.make_addplot(upper, type='line', color='b', secondary_y=True),
            mpf.make_addplot(lower, type='line', color='b', secondary_y=True),

        ]

In [28]:
mpf.plot(df, type='candle',
                        addplot=add_plot,
                        volume=False,
                        figscale=1.5,
                        title='Candle', figratio=(5, 5), ylabel='price', ylabel_lower='volume')

D:\Tools\Anaconda3\envs\elegantrl\lib\site-packages\mplfinance\_arg_validators.py:83: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+


In [17]:
df['exp12'] = exp12

In [19]:
df['exp26'] = exp26
df['macd'] = macd
df['signal'] = signal

In [24]:
import talib
upper, mid, lower = talib.BBANDS(df.Close, timeperiod=10, nbdevup=2, nbdevdn=2, matype=0)

In [29]:
df['upper'] = upper
df['mid'] = mid
df['lower'] = lower

In [13]:
df

,time,open,high,low,close,symbol
0,2022-11-01 00:00:00+00:00,0.98827,0.98827,0.98809,0.98809,EURUSD
1,2022-11-01 00:01:00+00:00,0.98808,0.98820,0.98808,0.98808,EURUSD
2,2022-11-01 00:02:00+00:00,0.98809,0.98821,0.98808,0.98812,EURUSD
3,2022-11-01 00:03:00+00:00,0.98812,0.98827,0.98812,0.98817,EURUSD
4,2022-11-01 00:04:00+00:00,0.98817,0.98820,0.98817,0.98818,EURUSD
...,...,...,...,...,...,...
5688,2022-11-04 22:50:00+00:00,0.99593,0.99609,0.99563,0.99587,EURUSD
5689,2022-11-04 22:51:00+00:00,0.99586,0.99602,0.99586,0.99599,EURUSD
5690,2022-11-04 22:52:00+00:00,0.99600,0.99603,0.99582,0.99586,EURUSD
5691,2022-11-04 22:53:00+00:00,0.99583,0.99598,0.99583,0.99593,EURUSD


In [20]:
from finrl.meta.data_processors.fx_history_data.vo import BarData
import pandas as pd
from typing import List
from finrl.meta.data_processors.fx_history_data.constant import Interval, Exchange

def df_to_vo(df: pd.DataFrame) -> List[BarData]:
    bars = []

    for _,row in df.iterrows():
        bar = BarData(
            exchange = Exchange.MT4,
            gateway_name = "DB",
            volume = 0,
            open_interest = 0,
            turnover = 0,
            close = row['close'],
            open = row['open'],
            high = row['high'],
            low = row['low'],
            time = row['time'],
            symbol = row['symbol']
        )

        bars.append(bar)
    return bars

In [33]:
from finrl.meta.data_processors.fx_history_data.bargenerator import BarGenerator
bar = BarGenerator(on_bar=df)

In [21]:
raw = df_to_vo(df)

In [8]:
from finrl.meta.data_processors.fx_history_data.bargenerator import BarGenerator
from finrl.meta.data_processors.fx_history_data.techfeatures import ArrayManager
from typing import Callable

In [ ]:
class Bars():
    def __init__(self, bar_data):
        """"""
        # daybar 生成器
        self.bar_data = bar_data

        self.m30_bar_gen = BarGenerator(self.on_bar, interval=Interval.MINUTE, window=30,on_window_bar=self.on_m30_bar)
        self.m30_am = ArrayManager(size=6)
        self.output_bars: List = [BarData]
        self.load_bar(20)

    def on_m30_bar(self, bar):
        self.m30_am.update_bar(bar)

    def on_bar(self, bar: BarData):
        self.m30_bar_gen.update_bar(bar)
        self.output_bars.append(self.m30_bar_gen.bar)

    def load_bar(
        self,
        rolling: int = 10,
        callback: Callable = None,
    ) -> None:

        if not callback:
            callback: Callable = self.on_bar

        bars: List[BarData] = self.bar_data

        for bar in bars:
            callback(bar)



In [23]:
raw[3].time

Timestamp('2022-11-01 00:03:00+0000', tz='UTC')

In [25]:
raw[10:]

[BarData(gateway_name='DB', extra=None, symbol='EURUSD', exchange=<Exchange.MT4: 'METATRADER4'>, time=Timestamp('2022-11-01 00:10:00+0000', tz='UTC'), interval=None, volume=0, turnover=0, open_interest=0, open=0.9882, high=0.9883, low=0.9882, close=0.98829),
 BarData(gateway_name='DB', extra=None, symbol='EURUSD', exchange=<Exchange.MT4: 'METATRADER4'>, time=Timestamp('2022-11-01 00:11:00+0000', tz='UTC'), interval=None, volume=0, turnover=0, open_interest=0, open=0.9883, high=0.98841, low=0.98829, close=0.98834),
 BarData(gateway_name='DB', extra=None, symbol='EURUSD', exchange=<Exchange.MT4: 'METATRADER4'>, time=Timestamp('2022-11-01 00:12:00+0000', tz='UTC'), interval=None, volume=0, turnover=0, open_interest=0, open=0.98835, high=0.98835, low=0.9883, close=0.98831),
 BarData(gateway_name='DB', extra=None, symbol='EURUSD', exchange=<Exchange.MT4: 'METATRADER4'>, time=Timestamp('2022-11-01 00:13:00+0000', tz='UTC'), interval=None, volume=0, turnover=0, open_interest=0, open=0.98831, 